In [172]:
#!pip install pyspark==3.0.1 py4j==0.10.9
from pyspark.sql import SparkSession

In [173]:
sp = SparkSession.builder\
        .master("local[6]")\
        .appName('PySpark_Lab7')\
        .getOrCreate()

In [174]:
# 1. Прочитайте CSV файл spark.read.csv()
from pyspark.sql.types import *
data_schema = [
    StructField('Name', StringType(), True), 
    StructField('Author', StringType(), True), 
    StructField('User rating', FloatType(), True), 
    StructField('Reviews', IntegerType(), True), 
    StructField('Price', IntegerType(), True), 
    StructField('Year', IntegerType(), True), 
    StructField('Genre', StringType(), True)
]
final_struct = StructType(fields = data_schema)
data = sp.read.csv("bestsellers with categories.csv", header = True, schema = final_struct)

In [175]:
# 2. Вивести схему даних за допомогою PrintSchema.
data.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Author: string (nullable = true)
 |-- User rating: float (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Genre: string (nullable = true)



In [176]:
# 3. Вивести назви стовпців і типи даних за допомогою dtypes
data.dtypes

[('Name', 'string'),
 ('Author', 'string'),
 ('User rating', 'float'),
 ('Reviews', 'int'),
 ('Price', 'int'),
 ('Year', 'int'),
 ('Genre', 'string')]

In [177]:
# 4. Виведіть перші п'ять рядків (використовується метод head)
data.head(5)

[Row(Name='10-Day Green Smoothie Cleanse', Author='JJ Smith', User rating=4.699999809265137, Reviews=17350, Price=8, Year=2016, Genre='Non Fiction'),
 Row(Name='11/22/63: A Novel', Author='Stephen King', User rating=4.599999904632568, Reviews=2052, Price=22, Year=2011, Genre='Fiction'),
 Row(Name='12 Rules for Life: An Antidote to Chaos', Author='Jordan B. Peterson', User rating=4.699999809265137, Reviews=18979, Price=15, Year=2018, Genre='Non Fiction'),
 Row(Name='1984 (Signet Classics)', Author='George Orwell', User rating=4.699999809265137, Reviews=21424, Price=6, Year=2017, Genre='Fiction'),
 Row(Name='5,000 Awesome Facts (About Everything!) (National Geographic Kids)', Author='National Geographic Kids', User rating=4.800000190734863, Reviews=7665, Price=12, Year=2019, Genre='Non Fiction')]

In [178]:
# 5. Вивести перший рядок за допомогою first()
data.first()

Row(Name='10-Day Green Smoothie Cleanse', Author='JJ Smith', User rating=4.699999809265137, Reviews=17350, Price=8, Year=2016, Genre='Non Fiction')

In [179]:
# 6. Зробіть опис датасету за допомогою describe()
data.describe()

DataFrame[summary: string, Name: string, Author: string, User rating: string, Reviews: string, Price: string, Year: string, Genre: string]

In [180]:
# 7. Виведіть список стовпців за допомогою columns()
data.columns

['Name', 'Author', 'User rating', 'Reviews', 'Price', 'Year', 'Genre']

In [181]:
# 8. Виведіть загальну кількість рядків даних за допомогою count()
print("Загальна кількість рядків датасету:", data.count())

Загальна кількість рядків датасету: 550


In [182]:
# 9. Перевірте, чи у всіх рядків вистачає даних. Якщо є пропуски, замінить їх на NaN, потім тип даних на числовий.
from pyspark.sql.functions import isnan, when, count, col

data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+----+------+-----------+-------+-----+----+-----+
|Name|Author|User rating|Reviews|Price|Year|Genre|
+----+------+-----------+-------+-----+----+-----+
|   0|     0|          0|      0|    0|   0|    0|
+----+------+-----------+-------+-----+----+-----+



In [183]:
# 10.Замініть комірки з пропущеними значенями на середні за стовпцем. І тип даних на float.
# Пропущених значень у датасеті немає.
# Про скільки книг зберігає дані датасет?
print("Датасет зберігає дані про", data.count(), "книг.")

Датасет зберігає дані про 550 книг.


In [184]:
# 11. Змініть назви всіх колонок: назва з маленької букви, без пробілів через підкреслення.
datar = data.toDF('name', 'author', 'user_rating', 
                  'reviews', 'price', 'year', 'genre')
datar.show()

+--------------------+--------------------+-----------+-------+-----+----+-----------+
|                name|              author|user_rating|reviews|price|year|      genre|
+--------------------+--------------------+-----------+-------+-----+----+-----------+
|10-Day Green Smoo...|            JJ Smith|        4.7|  17350|    8|2016|Non Fiction|
|   11/22/63: A Novel|        Stephen King|        4.6|   2052|   22|2011|    Fiction|
|12 Rules for Life...|  Jordan B. Peterson|        4.7|  18979|   15|2018|Non Fiction|
|1984 (Signet Clas...|       George Orwell|        4.7|  21424|    6|2017|    Fiction|
|5,000 Awesome Fac...|National Geograph...|        4.8|   7665|   12|2019|Non Fiction|
|A Dance with Drag...| George R. R. Martin|        4.4|  12643|   11|2011|    Fiction|
|A Game of Thrones...| George R. R. Martin|        4.7|  19735|   30|2014|    Fiction|
|A Gentleman in Mo...|         Amor Towles|        4.7|  19699|   15|2017|    Fiction|
|A Higher Loyalty:...|         James Comey|

In [221]:
# 13. Знайдіть автора з найвищим рейтингом.
from pyspark.sql.functions import expr
from pyspark.sql.functions import avg
max_user_rating = datar.agg({"user_rating": "max"}).collect()[0][0]
max_user_rating_rows = datar.filter(expr(f"user_rating = {max_user_rating}"))

print("Автори з найвищими рейтингами:")
max_user_rating_rows.select(['author', 
                             'user_rating']).groupBy("author").agg(avg("user_rating").alias("AverageRating")).show()

Автори з найвищими рейтингами:
+--------------------+-----------------+
|              author|    AverageRating|
+--------------------+-----------------+
|         Sarah Young|4.900000095367432|
|          Jill Twiss|4.900000095367432|
|       Rush Limbaugh|4.900000095367432|
|         Chip Gaines|4.900000095367432|
|       Mark R. Levin|4.900000095367432|
|         Jeff Kinney|4.900000095367432|
|      Nathan W. Pyle|4.900000095367432|
|  Lin-Manuel Miranda|4.900000095367432|
|          Dav Pilkey|4.900000095367432|
|           Dr. Seuss|4.900000095367432|
|      Alice Schertle|4.900000095367432|
|     Bill Martin Jr.|4.900000095367432|
|      Patrick Thorpe|4.900000095367432|
|Emily Winfield Ma...|4.900000095367432|
|          Pete Souza|4.900000095367432|
|     Brandon Stanton|4.900000095367432|
|       J. K. Rowling|4.900000095367432|
|          Eric Carle|4.900000095367432|
|Sherri Duskey Rinker|4.900000095367432|
|        J.K. Rowling|4.900000095367432|
+--------------------+----

In [220]:
# 14. Знайдіть автора з найнижчим рейтингом.
min_user_rating = datar.agg({"user_rating": "min"}).collect()[0][0]
min_user_rating_rows = datar.filter(expr(f"user_rating = {min_user_rating}"))

print("Автори з найнижчими рейтингами:")
min_user_rating_rows.select(['author', 
                             'user_rating']).groupBy("author").agg(avg("user_rating").alias("AverageRating")).show()

Автори з найнижчими рейтингами:
+------------+-----------------+
|      author|    AverageRating|
+------------+-----------------+
|J.K. Rowling|3.299999952316284|
+------------+-----------------+



In [224]:
# 15. Знайдіть автора, що має найбільшу кількіть рецензій.
max_user_reviews = datar.agg({"reviews": "max"}).collect()[0][0]
max_user_reviews_rows = datar.filter(expr(f"reviews = {max_user_reviews}"))

print("Автори з найбільшими кількостями рецензій:")
max_user_reviews_rows.select(['author', 
                             'reviews']).groupBy("author").agg(avg("reviews").alias("AverageReviews")).show()

Автори з найбільшими кількостями рецензій:
+-----------+--------------+
|     author|AverageReviews|
+-----------+--------------+
|Delia Owens|       87841.0|
+-----------+--------------+



In [273]:
# 16. Побудуйте гістограму рейтингу 10 найкращих книг (відібрати книги з найбільшим рейтингом).
import plotly.express as px
user_rating_wd = max_user_rating_rows.distinct().select(["name", "user_rating"]).toPandas()
user_rating_wd10 = user_rating_wd[1:12]
user_rating_wd10.drop_duplicates(inplace = True)

fig = px.histogram(user_rating_wd10, x = "name", y = "user_rating", template = "plotly_dark", 
                   title = "10 best books by rating", labels={
                     "name": "Names of books",
                     "user_rating": "Ratings books"})
fig.show()

C:\Users\User\AppData\Local\Temp/ipykernel_12692/2905245349.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [283]:
# 17. Побудуйте матрицю кореляції для стовпчиків 'user_rating', 'reviews', 'price'.
data_matrix_wd = max_user_rating_rows.distinct().select(["user_rating", "reviews", "price"]).toPandas()
convert_dtypes = {'user_rating': float, 'reviews': int, 'price': int}
data_matrix_wd = data_matrix_wd.astype(convert_dtypes)
matrix = data_matrix_wd.corr()
matrix

,user_rating,reviews,price
user_rating,NaN,NaN,NaN
reviews,NaN,1.000000,-0.287078
price,NaN,-0.287078,1.000000
